## Multi Label Classification of Stack Overflow tags - basis of what we are trying to achieve

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/stackoverflow.csv', index_col=0)

In [3]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"['sql', 'asp.net']"
4,adding scripting functionality to net applicat...,"['c#', '.net']"
5,should i use nested classes in this case i am ...,['c++']
6,homegrown consumption of web services i have b...,['.net']
8,automatically update version number i would li...,['c#']


### Tags from STRING to LIST uses the AST library for better processing of the tags

In [4]:
import ast

In [5]:
df['Tags'].iloc[0]

"['sql', 'asp.net']"

In [6]:
ast.literal_eval(df.Tags.iloc[0])

['sql', 'asp.net']

In [7]:
df['Tags'] = df['Tags'].apply(lambda x : ast.literal_eval(x))

In [8]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"[sql, asp.net]"
4,adding scripting functionality to net applicat...,"[c#, .net]"
5,should i use nested classes in this case i am ...,[c++]
6,homegrown consumption of web services i have b...,[.net]
8,automatically update version number i would li...,[c#]


### Predictive ML part

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier

In [10]:
y = df['Tags']

In [11]:
y.head()

2    [sql, asp.net]
4        [c#, .net]
5             [c++]
6            [.net]
8              [c#]
Name: Tags, dtype: object

In [12]:
multilabel = MultiLabelBinarizer()

In [13]:
y = multilabel.fit_transform(df['Tags'])

In [14]:
y

array([[0, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
multilabel.classes_

array(['.net', 'android', 'asp.net', 'c', 'c#', 'c++', 'css', 'html',
       'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql',
       'objective-c', 'php', 'python', 'ruby', 'ruby-on-rails', 'sql'],
      dtype=object)

In [16]:
pd.DataFrame(y, columns=multilabel.classes_)

,.net,android,asp.net,c,c#,c++,css,html,ios,iphone,java,javascript,jquery,mysql,objective-c,php,python,ruby,ruby-on-rails,sql
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48971,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48972,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48973,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [17]:
tfidf = TfidfVectorizer(analyzer='word', max_features=10000, ngram_range=(1,2))

In [18]:
X = tfidf.fit_transform(df['Text'])

In [19]:
X.shape

(48976, 10000)

In [20]:
y.shape

(48976, 20)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Build Model

In [22]:
sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()

In [23]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis=1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100

def print_score(y_pred, clf):
    print("Clf : ", clf.__class__.__name__)
    print('Jacard score: {}'.format(j_score(y_test, y_pred)))
    print('----')

In [24]:
for classifier in [sgd, lr, svc] :
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

Clf :  SGDClassifier
Jacard score: 48.70950728188376
----


/Users/sidchiku9/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Clf :  LogisticRegression
Jacard score: 47.757928406152175
----
Clf :  LinearSVC
Jacard score: 60.01224989791753
----


In [ ]:
### testing with data

### Real time testing

In [25]:
x = [ 'how to write ml code in python and java i have data but do not know how to do it']

In [26]:
xt = tfidf.transform(x)

In [27]:
clf.predict(xt)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [28]:
multilabel.inverse_transform(clf.predict(xt))

[('java', 'python')]

In [29]:
y = [ 'homegrown consumption of web services i have been writing a few web services for a net app now i am ready to consume them i have seen numerous examples where there is homegrown code for consuming the service as opposed to using the auto generated methods visual studio creates when adding the web reference is there some advantage to this']

In [30]:
yt = tfidf.transform(y)
clf.predict(yt)
multilabel.inverse_transform(clf.predict(yt))

[('.net', 'c#')]